In [1]:
library(lmtp)

Major changes in lmtp 0.9.0. Consult NEWS.md for more information.



In [2]:
set.seed(56)

n <- 1000
W <- rnorm(n, 10, 5)
A <- 23 + 5 * W + rnorm(n)
Y <- 7.2 * A + 3 * W + rnorm(n)

In [3]:
ex1_dat <- data.frame(W, A, Y)
d <- function(data, x) data[[x]] - 5
psi1.1 <- lmtp_tmle(ex1_dat, "A", "Y", "W", shift = d,
                    outcome_type = "continuous", folds = 2)
psi1.1

Loading required package: nnls

Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: algorithm did not converge"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1

In [4]:
d <- function(data, x) (data[[x]] > 80) * (data[[x]] - 15) + (data[[x]] <= 80) * data[[x]]
psi1.2 <- lmtp_tmle(ex1_dat, "A", "Y", "W", shift = d,
                   outcome_type = "continuous", folds = 2)
psi1.2

LMTP Estimator: TMLE

Trt. Policy: (d)

Population intervention effect

Estimate: 507.7651

Std. error: 4.8914

95% CI: (498.1782, 517.352)



In [5]:
head(sim_t4)

,ID,L_1,A_1,L_2,A_2,L_3,A_3,L_4,A_4,Y
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,2,3,0,1,1,1,1,3,0
2,2,2,1,1,4,0,3,1,2,0
3,3,1,0,1,3,1,2,1,1,1
4,4,1,0,0,3,1,3,1,2,0
5,5,3,3,1,1,0,1,1,2,0
6,6,1,0,0,2,0,3,1,4,0


In [6]:
a <- c("A_1", "A_2", "A_3", "A_4")
tv <- list(c("L_1"), c("L_2"), c("L_3"), c("L_4"))
d <- function(data, trt) {
    a <- data[[trt]]
    (a - 1) * (a - 1 >= 1) + a * (a - 1 < 1)
}

progressr::with_progress({
    psi2.1 <- lmtp_tmle(sim_t4, a, "Y", time_vary = tv,
                        shift = d, folds = 2)
})
psi2.1

LMTP Estimator: TMLE

Trt. Policy: (d)

Population intervention effect

Estimate: 0.2646

Std. error: 0.0173

95% CI: (0.2308, 0.2984)



In [7]:
progressr::with_progress({
    psi2.2 <- lmtp_tmle(sim_t4, a, "Y", time_vary = tv,
                       shift = d, folds = 2, k = 0)
})
psi2.2

LMTP Estimator: TMLE

Trt. Policy: (d)

Population intervention effect

Estimate: 0.2626

Std. error: 0.0179

95% CI: (0.2275, 0.2977)



In [8]:
a <- c("A_1", "A_2", "A_3", "A_4")
time_varying <- list(c("L_1"), c("L_2"), c("L_3"), c("L_4"))
shift <- function(data, trt) {
    (data[[trt]] - 1) * (data[[trt]] - 1 >= 1) + data[[trt]] * (data[[trt]] - 1 < 1)
}
dynamic_mtp <- function(data, trt) {
    if (trt == "A_1") {
        shift(data, trt)
    } else {
        ifelse(data[[sub("A", "L", trt)]] == 1,
               shift(data, trt),
               data[[trt]])
    }
}
psi2.3 <- lmtp_tmle(sim_t4, a, "Y", time_vary = time_varying,
                    k = 0, shift = dynamic_mtp, folds = 2)
psi2.3

LMTP Estimator: TMLE

Trt. Policy: (dynamic_mtp)

Population intervention effect

Estimate: 0.2367

Std. error: 0.0114

95% CI: (0.2144, 0.259)



In [9]:
for (i in a) {
    sim_t4[[i]] <- factor(sim_t4[[i]], levels = 0:5, ordered = TRUE)
}

d <- function(data, trt) {
    out <- list()
    a <- data[[trt]]
    for (i in 1:length(a)) {
        if (as.character(a[i]) %in% c("0", "1")) {
            out[[i]] <- as.character(a[i])
        } else {
            out[[i]] <- as.numeric(as.character(a[i])) - 1
        }
    }
    factor(unlist(out), levels = 0:5, ordered = TRUE)
}

progressr::with_progress({
    psi2.4 <- lmtp_tmle(sim_t4, a, "Y", time_vary = tv,
                        shift = d, k = 0, folds = 2)
})
psi2.4

Warning message:
"Some of your variables appear to be factors. Make sure your SuperLearner library is capable of handling factors!"
LMTP Estimator: TMLE

Trt. Policy: (d)

Population intervention effect

Estimate: 0.3026

Std. error: 0.0183

95% CI: (0.2667, 0.3384)



In [10]:
library(twang)
data(iptwExWide)

a <- paste0("tx", 1:3)
baseline <- c("gender", "age")
tv <- list(c("use0"), c("use1"), c("use2"))
progressr::with_progress({
    psi3.1 <- lmtp_tmle(iptwExWide, a, "outcome", baseline = baseline, time_vary = tv,
        shift = static_binary_on, outcome_type = "continuous", folds = 2)
})
psi3.1

To reproduce results from prior versions of the twang package, please see the version="legacy" option described in the documentation.

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
W

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misl

Estimate: -0.2889

Std. error: 0.0431

95% CI: (-0.3733, -0.2045)



In [11]:
head(sim_cens)

,L1,A1,C1,L2,A2,C2,Y
,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>
1,0,0.7930531,1,0,0.4098544,1,0
2,1,3.9568969,1,1,7.1306144,1,1
3,0,-0.9803504,1,0,-2.7858973,1,0
4,1,0.8833268,1,1,1.1372421,1,1
5,1,3.1666751,1,1,7.4640462,1,1
6,1,3.5393771,1,1,9.5161813,1,1


In [12]:
a <- c("A1", "A2")
tv <- list(c("L1"), c("L2"))
cens <- c("C1", "C2")
y <- "Y"
psi4.1 <- lmtp_tmle(sim_cens, a, y, time_vary = tv, cens = cens,
                    shift = NULL, folds = 2)
psi4.1

LMTP Estimator: TMLE

Trt. Policy: (NULL)

Population intervention effect

Estimate: 0.7973

Std. error: 0.0129

95% CI: (0.772, 0.8227)



In [13]:
d <- function(data, x) data[[x]] + 0.5
psi4.2 <- lmtp_tmle(sim_cens, a, y, time_vary = tv, cens = cens,
                    shift = d, folds = 2)
psi4.2

LMTP Estimator: TMLE

Trt. Policy: (d)

Population intervention effect

Estimate: 0.8986

Std. error: 0.0094

95% CI: (0.8801, 0.917)



In [14]:
a <- "trt"
y <- paste0("Y.", 1:6)
cens <- paste0("C.", 0:5)
baseline <- c("W1", "W2")
progressr::with_progress({
    psi5.1 <- lmtp_tmle(sim_point_surv, a, y, baseline, cens = cens,
                        shift = static_binary_on, folds = 2,
                        outcome_type = "survival")
})
psi5.1

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misl

In [15]:
baseline <- c("W_1", "W_2")
trt <- c("A_1", "A_2")
time_vary <- list(c("L_11", "L_12"), 
                  c("L_21", "L_22"))
create_node_list(trt = trt, baseline = baseline, time_vary = time_vary, tau = 2)

$trt
$trt[[1]]
[1] "W_1"  "W_2"  "L_11" "L_12" "A_1" 

$trt[[2]]
[1] "W_1"  "W_2"  "L_11" "L_12" "L_21" "L_22" "A_1"  "A_2" 


$outcome
$outcome[[1]]
[1] "W_1"  "W_2"  "L_11" "L_12" "A_1" 

$outcome[[2]]
[1] "W_1"  "W_2"  "L_11" "L_12" "A_1"  "L_21" "L_22" "A_2"

In [16]:
shift <- function(data, trt) {
  (data[[trt]] - 1) * (data[[trt]] - 1 >= 1) + data[[trt]] * (data[[trt]] - 1 < 1)
}

In [17]:
A <- c("A_1", "A_2", "A_3", "A_4")
L <- list(c("L_1"), c("L_2"), c("L_3"), c("L_4"))
lmtp_sdr(sim_t4, A, "Y", time_vary = L, k = 0, shift = shift, folds = 5)

Warning message:
"Some of your variables appear to be factors. Make sure your SuperLearner library is capable of handling factors!"
Warning message in Ops.ordered(data[[trt]], 1):
"'-' is not meaningful for ordered factors"
Warning message in Ops.ordered(data[[trt]], 1):
"'-' is not meaningful for ordered factors"
Warning message in Ops.ordered(data[[trt]], 1):
"'-' is not meaningful for ordered factors"
Warning message in Ops.ordered(data[[trt]], (data[[trt]] - 1 < 1)):
"'*' is not meaningful for ordered factors"
Warning message in Ops.ordered(data[[trt]], 1):
"'-' is not meaningful for ordered factors"
Warning message in Ops.ordered(data[[trt]], 1):
"'-' is not meaningful for ordered factors"
Warning message in Ops.ordered(data[[trt]], 1):
"'-' is not meaningful for ordered factors"
Warning message in Ops.ordered(data[[trt]], (data[[trt]] - 1 < 1)):
"'*' is not meaningful for ordered factors"
Warning message in Ops.ordered(data[[trt]], 1):
"'-' is not meaningful for ordered factors"


ERROR: Error in SuperLearner::SuperLearner(Y, X, family = family[[1]], SL.library = learners, : missing data is currently not supported. Check Y, X, and newX for missing values
